<a href="https://colab.research.google.com/github/ROARMarketingConcepts/PySpark-Projects/blob/main/PySpark_Dataframe_Manipulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PySpark Dataframe Manipulations

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark import SparkConf, SparkContext
from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("PySpark Book Ch. 3") \
       .getOrCreate()

spark

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,926 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,720 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,901

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Read in the datasets

In [ ]:
file_path = '/content/drive/MyDrive/Colab_Notebooks/PySpark Data Analysis/Advanced Analytics With PySpark - OReilly/profiledata_06-May-2005/'

raw_user_artist_data = spark.read.text(file_path+'user_artist_data.txt')
raw_user_artist_data.show()

+-------------------+
|              value|
+-------------------+
|       1000002 1 55|
| 1000002 1000006 33|
|  1000002 1000007 8|
|1000002 1000009 144|
|1000002 1000010 314|
|  1000002 1000013 8|
| 1000002 1000014 42|
| 1000002 1000017 69|
|1000002 1000024 329|
|  1000002 1000025 1|
| 1000002 1000028 17|
| 1000002 1000031 47|
| 1000002 1000033 15|
|  1000002 1000042 1|
|  1000002 1000045 1|
|  1000002 1000054 2|
| 1000002 1000055 25|
|  1000002 1000056 4|
|  1000002 1000059 2|
| 1000002 1000062 71|
+-------------------+
only showing top 20 rows



In [ ]:
raw_artist_data = spark.read.text(file_path+'artist_data.txt')
raw_artist_data.show()

+--------------------+
|               value|
+--------------------+
|1134999\t06Crazy ...|
|6821360\tPang Nak...|
|10113088\tTerfel,...|
|10151459\tThe Fla...|
|6826647\tBodensta...|
|10186265\tJota Qu...|
|6828986\tToto_XX ...|
|10236364\tU.S Bom...|
|1135000\tartist f...|
|10299728\tKassier...|
|10299744\tRahzel,...|
|6864258\tJon Rich...|
|6878791\tYoung Fr...|
|10299751\tKi-ya-Kiss|
|6909716\tUndermin...|
|   10435121\tKox-Box|
|6918061\talexison...|
|1135001\tdj salinger|
|6940391\tThe B52'...|
|   10475396\t44 Hoes|
+--------------------+
only showing top 20 rows



In [ ]:
raw_artist_alias = spark.read.text(file_path+'artist_alias.txt')
raw_artist_alias.show()

+-----------------+
|            value|
+-----------------+
| 1092764\t1000311|
| 1095122\t1000557|
| 6708070\t1007267|
|10088054\t1042317|
| 1195917\t1042317|
| 1112006\t1000557|
| 1187350\t1294511|
| 1116694\t1327092|
| 6793225\t1042317|
| 1079959\t1000557|
| 6789612\t1000591|
| 1262241\t1000591|
| 6791455\t1000591|
| 6694867\t1000591|
|10141141\t1113738|
| 1295140\t1000591|
| 1027859\t1252408|
| 2127019\t1000591|
| 2153974\t1000591|
| 1232342\t1000591|
+-----------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.types import IntegerType, StringType

user_artist_df = raw_user_artist_data.withColumn('user',
                                    F.split(raw_user_artist_data['value'], ' ').\
                                    getItem(0).\
                                    cast(IntegerType()))
user_artist_df = user_artist_df.withColumn('artist',
                                    F.split(raw_user_artist_data['value'], ' ').\
                                    getItem(1).\
                                    cast(IntegerType()))
user_artist_df = user_artist_df.withColumn('count',
                                    F.split(raw_user_artist_data['value'], ' ').\
                                    getItem(2).\
                                    cast(IntegerType())).drop('value')

user_artist_df.select([F.min("user"), F.max("user"), F.min("artist"),F.max("artist")]).show()

+---------+---------+-----------+-----------+
|min(user)|max(user)|min(artist)|max(artist)|
+---------+---------+-----------+-----------+
|       90|  2443548|          1|   10794401|
+---------+---------+-----------+-----------+



In [ ]:
user_artist_df.show()

+-------+-------+-----+
|   user| artist|count|
+-------+-------+-----+
|1000002|      1|   55|
|1000002|1000006|   33|
|1000002|1000007|    8|
|1000002|1000009|  144|
|1000002|1000010|  314|
|1000002|1000013|    8|
|1000002|1000014|   42|
|1000002|1000017|   69|
|1000002|1000024|  329|
|1000002|1000025|    1|
|1000002|1000028|   17|
|1000002|1000031|   47|
|1000002|1000033|   15|
|1000002|1000042|    1|
|1000002|1000045|    1|
|1000002|1000054|    2|
|1000002|1000055|   25|
|1000002|1000056|    4|
|1000002|1000059|    2|
|1000002|1000062|   71|
+-------+-------+-----+
only showing top 20 rows



In [ ]:
artist_by_id_df = raw_artist_data.withColumn('id',
                                    F.split(raw_artist_data['value'], '\t').\
                                    getItem(0).\
                                    cast(IntegerType()))
artist_by_id_df = artist_by_id_df.withColumn('artist',
                                    F.split(raw_artist_data['value'], '\t').\
                                    getItem(1)).drop('value')

artist_by_id_df.show()

+--------+--------------------+
|      id|              artist|
+--------+--------------------+
| 1134999|        06Crazy Life|
| 6821360|        Pang Nakarin|
|10113088|Terfel, Bartoli- ...|
|10151459| The Flaming Sidebur|
| 6826647|   Bodenstandig 3000|
|10186265|Jota Quest e Ivet...|
| 6828986|       Toto_XX (1977|
|10236364|         U.S Bombs -|
| 1135000|artist formaly kn...|
|10299728|Kassierer - Musik...|
|10299744|         Rahzel, RZA|
| 6864258|      Jon Richardson|
| 6878791|Young Fresh Fello...|
|10299751|          Ki-ya-Kiss|
| 6909716|Underminded - The...|
|10435121|             Kox-Box|
| 6918061|  alexisonfire [wo!]|
| 1135001|         dj salinger|
| 6940391|The B52's - Chann...|
|10475396|             44 Hoes|
+--------+--------------------+
only showing top 20 rows



In [ ]:
artist_alias_df = raw_artist_alias.withColumn('id', F.split(raw_artist_alias['value'], '\t').getItem(0).cast(IntegerType()))
artist_alias_df = artist_alias_df.withColumn('alias', F.split(raw_artist_alias['value'], '\t').getItem(1).cast(IntegerType())).drop('value')

artist_alias_df.show()

+--------+-------+
|      id|  alias|
+--------+-------+
| 1092764|1000311|
| 1095122|1000557|
| 6708070|1007267|
|10088054|1042317|
| 1195917|1042317|
| 1112006|1000557|
| 1187350|1294511|
| 1116694|1327092|
| 6793225|1042317|
| 1079959|1000557|
| 6789612|1000591|
| 1262241|1000591|
| 6791455|1000591|
| 6694867|1000591|
|10141141|1113738|
| 1295140|1000591|
| 1027859|1252408|
| 2127019|1000591|
| 2153974|1000591|
| 1232342|1000591|
+--------+-------+
only showing top 20 rows



In [ ]:
artist_by_id_df.filter(artist_by_id_df.id.isin(1092764, 1000311)).show()

+-------+--------------+
|     id|        artist|
+-------+--------------+
|1000311| Steve Winwood|
|1092764|Winwood, Steve|
+-------+--------------+



To join `user_artist_df` with `artist_alias_df`, we broadcast the `artist_alias_df` DataFrame created earlier. This makes Spark send and hold in memory just one copy for each executor in the cluster. It's important to note that broadcasting is most effective when one DataFrame is significantly smaller than the other.

In [ ]:
from pyspark.sql.functions import broadcast, when

train_data = user_artist_df.join(broadcast(artist_alias_df), user_artist_df.artist == artist_alias_df.id, how='left')
train_data.show()

+-------+-------+-----+----+-----+
|   user| artist|count|  id|alias|
+-------+-------+-----+----+-----+
|1000002|      1|   55|NULL| NULL|
|1000002|1000006|   33|NULL| NULL|
|1000002|1000007|    8|NULL| NULL|
|1000002|1000009|  144|NULL| NULL|
|1000002|1000010|  314|NULL| NULL|
|1000002|1000013|    8|NULL| NULL|
|1000002|1000014|   42|NULL| NULL|
|1000002|1000017|   69|NULL| NULL|
|1000002|1000024|  329|NULL| NULL|
|1000002|1000025|    1|NULL| NULL|
|1000002|1000028|   17|NULL| NULL|
|1000002|1000031|   47|NULL| NULL|
|1000002|1000033|   15|NULL| NULL|
|1000002|1000042|    1|NULL| NULL|
|1000002|1000045|    1|NULL| NULL|
|1000002|1000054|    2|NULL| NULL|
|1000002|1000055|   25|NULL| NULL|
|1000002|1000056|    4|NULL| NULL|
|1000002|1000059|    2|NULL| NULL|
|1000002|1000062|   71|NULL| NULL|
+-------+-------+-----+----+-----+
only showing top 20 rows



Making sure our `join` worked as expected...

In [ ]:
value_counts_df = train_data.groupBy("id").count().orderBy(F.col("count").desc())
value_counts_df.show()

+-------+--------+
|     id|   count|
+-------+--------+
|   NULL|23878789|
|     15|   22911|
|    709|   17683|
|    754|   14969|
|1001864|    9272|
|   5702|    8232|
|1014609|    5291|
|1006322|    5268|
|1000129|    3790|
|1002498|    2967|
|1006102|    2558|
|1004580|    2291|
|1008798|    2234|
|1018110|    1787|
|1002139|    1761|
|1008455|    1474|
|2075188|    1353|
|1000110|    1145|
|1003801|    1117|
|1014175|    1113|
+-------+--------+
only showing top 20 rows



Get artist’s alias if it exists; otherwise, get original artist.

In [ ]:
train_data = train_data.withColumn('artist',
                                    when(F.col('alias').isNull(), F.col('artist')).\
                                    otherwise(F.col('alias')))
train_data.show()

+-------+-------+-----+----+-----+
|   user| artist|count|  id|alias|
+-------+-------+-----+----+-----+
|1000002|      1|   55|NULL| NULL|
|1000002|1000006|   33|NULL| NULL|
|1000002|1000007|    8|NULL| NULL|
|1000002|1000009|  144|NULL| NULL|
|1000002|1000010|  314|NULL| NULL|
|1000002|1000013|    8|NULL| NULL|
|1000002|1000014|   42|NULL| NULL|
|1000002|1000017|   69|NULL| NULL|
|1000002|1000024|  329|NULL| NULL|
|1000002|1000025|    1|NULL| NULL|
|1000002|1000028|   17|NULL| NULL|
|1000002|1000031|   47|NULL| NULL|
|1000002|1000033|   15|NULL| NULL|
|1000002|1000042|    1|NULL| NULL|
|1000002|1000045|    1|NULL| NULL|
|1000002|1000054|    2|NULL| NULL|
|1000002|1000055|   25|NULL| NULL|
|1000002|1000056|    4|NULL| NULL|
|1000002|1000059|    2|NULL| NULL|
|1000002|1000062|   71|NULL| NULL|
+-------+-------+-----+----+-----+
only showing top 20 rows



In [ ]:
train_data.printSchema()

root
 |-- user: integer (nullable = true)
 |-- artist: integer (nullable = true)
 |-- count: integer (nullable = true)
 |-- id: integer (nullable = true)
 |-- alias: integer (nullable = true)



In [ ]:
train_data = train_data.withColumn('artist', F.col('artist').\
                                             cast(IntegerType())).\
                                             drop('alias')
train_data.show()

+-------+-------+-----+----+
|   user| artist|count|  id|
+-------+-------+-----+----+
|1000002|      1|   55|NULL|
|1000002|1000006|   33|NULL|
|1000002|1000007|    8|NULL|
|1000002|1000009|  144|NULL|
|1000002|1000010|  314|NULL|
|1000002|1000013|    8|NULL|
|1000002|1000014|   42|NULL|
|1000002|1000017|   69|NULL|
|1000002|1000024|  329|NULL|
|1000002|1000025|    1|NULL|
|1000002|1000028|   17|NULL|
|1000002|1000031|   47|NULL|
|1000002|1000033|   15|NULL|
|1000002|1000042|    1|NULL|
|1000002|1000045|    1|NULL|
|1000002|1000054|    2|NULL|
|1000002|1000055|   25|NULL|
|1000002|1000056|    4|NULL|
|1000002|1000059|    2|NULL|
|1000002|1000062|   71|NULL|
+-------+-------+-----+----+
only showing top 20 rows



In [ ]:
train_data.cache()
train_data.count()

24296858

In [ ]:
from pyspark.ml.recommendation import ALS

model = ALS(rank=10, seed=0, maxIter=5, regParam=0.1,
            implicitPrefs=True, alpha=1.0, userCol='user',
            itemCol='artist', ratingCol='count'). \
        fit(train_data)

In [ ]:
model.userFactors.show(1, truncate=False)

+---+------------------------------------------------------------------------------------------------------------------------------+
|id |features                                                                                                                      |
+---+------------------------------------------------------------------------------------------------------------------------------+
|90 |[0.16020624, 0.20717518, -0.17194684, 0.06038469, 0.062727705, 0.54658705, -0.40481892, 0.43657345, -0.10396775, -0.042728312]|
+---+------------------------------------------------------------------------------------------------------------------------------+
only showing top 1 row



In [ ]:
user_id = 2093760
existing_artist_ids = train_data.filter(train_data.user == user_id).select('artist').collect()
existing_artist_ids

[Row(artist=1180),
 Row(artist=1255340),
 Row(artist=378),
 Row(artist=813),
 Row(artist=942)]

In [ ]:
existing_artist_ids = [i[0] for i in existing_artist_ids]
existing_artist_ids

[1180, 1255340, 378, 813, 942]

In [ ]:
artist_by_id_df.filter(artist_by_id_df.id.isin(existing_artist_ids)).show()

+-------+---------------+
|     id|         artist|
+-------+---------------+
|   1180|     David Gray|
|    378|  Blackalicious|
|    813|     Jurassic 5|
|1255340|The Saw Doctors|
|    942|         Xzibit|
+-------+---------------+



In [ ]:
user_subset = train_data.select('user').where(F.col('user') == user_id).distinct()
user_subset.show()

+-------+
|   user|
+-------+
|2093760|
+-------+



In [ ]:
top_predictions = model.recommendForUserSubset(user_subset, 5)
top_predictions.show()

+-------+--------------------+
|   user|     recommendations|
+-------+--------------------+
|2093760|[{2814, 0.0294106...|
+-------+--------------------+



In [ ]:
top_predictions_exploded = top_predictions.select('user', F.explode('recommendations').alias('rec'))
top_predictions_exploded = top_predictions_exploded.select('user', 'rec.artist', 'rec.rating')
top_predictions_exploded.show()

+-------+-------+-----------+
|   user| artist|     rating|
+-------+-------+-----------+
|2093760|   2814|0.029410677|
|2093760|1300642|0.028404651|
|2093760|1001819|0.028333459|
|2093760|1007614|0.027992856|
|2093760|   4605|0.027890977|
+-------+-------+-----------+



In [ ]:
top_predictions_exploded.join(artist_by_id_df, top_predictions_exploded.artist == artist_by_id_df.id).drop('id').show()

+-------+-------+-----------+----------+
|   user| artist|     rating|    artist|
+-------+-------+-----------+----------+
|2093760|   2814|0.029410677|   50 Cent|
|2093760|   4605|0.027890977|Snoop Dogg|
|2093760|1007614|0.027992856|     Jay-Z|
|2093760|1001819|0.028333459|      2Pac|
|2093760|1300642|0.028404651|  The Game|
+-------+-------+-----------+----------+

